In [1]:
!pip install nbformat


In [2]:
import nbformat

nb = nbformat.read("Groq_RAG_App_with_Gradio.ipynb", as_version=4)

# Ensure widget metadata has the 'state' key
if "widgets" in nb["metadata"]:
    nb["metadata"]["widgets"].setdefault("state", {})

nbformat.write(nb, "Groq_RAG_App_with_Gradio_fixed.ipynb")


### RETRIEVAL AUGMENTED GENERATION [RAG]



### What is RAG?
One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as retrieval-augmented generation (RAG). RAG is a technique for augmenting LLM knowledge with additional data, which can be your own data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to public data up to the specific point in time that they were trained. If you want to build AI applications that can reason about private data or data introduced after a model’s cut-off date, you must augment the knowledge of the model with the specific information that it needs. The process of bringing and inserting the appropriate information into the model prompt is known as RAG.

LangChain has several components that are designed to help build Q&A applications and RAG applications, more generally.


### RAG architecture
A typical RAG application has two main components:

* **Indexing**: A pipeline for ingesting and indexing data from a source. This usually happens offline.

* **Retrieval and generation**: The actual RAG chain takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

### Importing Required Libraries

In [1]:
!pip install -U langchain-community
!pip install langchain-groq
!pip install wget

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=14e0fc5bcb3947625055147682602fd2280d38d11680cd1c1faf3232664d2a77
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [2]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq

import wget

### Preprocessing

Loading the document

In [3]:
file_name = 'The_Adventures_of_Sherlock_Holmes.txt'
url = 'https://www.gutenberg.org/ebooks/1661.txt.utf-8'

wget.download(url, out=file_name)
print('Book Downloaded')


Book Downloaded


In [4]:
with open(file_name, 'r') as file:
  content = file.read()
  print(content[:500])

The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before usin


### Splitting Document into Chunks

In [5]:
loader = TextLoader(file_name)
document = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30)
texts = text_splitter.split_documents(document)
print(len(texts))

687


### Embedding and storing
This step is the `embed` and `store` processes in `Indexing`.

In [6]:
!pip install chromadb
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
print('document ingested')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.8 MB/s e

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

document ingested


In [7]:
model_id = 'llama-3.1-8b-instant'

In [ ]:
llama_llm = ChatGroq(
    api_key = "YOUR_GROQ_API_KEY",
    model_name=model_id,
    temperature=0.5,
    max_tokens=256
)


### RetrievalQA

In [9]:
qa = RetrievalQA.from_chain_type(llm = llama_llm,
                                 chain_type = 'stuff',
                                 retriever = docsearch.as_retriever(),
                                 return_source_documents = False
                                 )
query = 'Can you summarize the document?'
qa.invoke(query)

{'query': 'Can you summarize the document?',
 'result': 'The document analyzed appears to be a personality profile or assessment of a person. It lists various aspects of their life and skills, including:\n\n- Philosophy: 0 (implying a lack of interest or knowledge)\n- Astronomy: 0\n- Politics: 0\n- Botany: variable\n- Geology: profound (with a focus on mud-stains within 50 miles of town)\n- Chemistry: eccentric\n- Anatomy: unsystematic\n- Sensational literature and crime records: unique\n- Violin-player: (implying they play the violin)\n- Boxer: (implying they are a skilled boxer)\n- Swordsman: (implying they are skilled with a sword)\n- Lawyer: (implying they have a background or interest in law)\n- Self-poisoner (by cocaine and tobacco): (implying they have a history of substance abuse)\n\nThe profile also mentions that the person is to copy out the first volume of the Encyclopædia Britannica.'}

In [10]:
qa = RetrievalQA.from_chain_type(llm = llama_llm,
                                 chain_type = 'stuff',
                                 retriever = docsearch.as_retriever(),
                                 return_source_documents = False
                                 )
query = 'Give me the important Charater names?'
qa.invoke(query)

{'query': 'Give me the important Charater names?',
 'result': "The important character names mentioned in the context are:\n\n1. Sherlock Holmes\n2. Lestrade (friend of Holmes and a detective)\n3. Mr. John Turner (a landed proprietor in Herefordshire)\n4. Mr. Charles McCarthy (Mr. Turner's tenant and an ex-Australian)\n5. Mr. Godfrey Norton (a Mr. Turner's male visitor and a lawyer from the Inner Temple)"}

In [11]:
qa = RetrievalQA.from_chain_type(llm = llama_llm,
                                 chain_type = 'stuff',
                                 retriever = docsearch.as_retriever(),
                                 return_source_documents = False
                                 )
query = 'Is there a movie based on the book?'
qa.invoke(query)

{'query': 'Is there a movie based on the book?',
 'result': 'Yes, there is a movie based on the book. The story is from a short story called "The Stone of Evil" but more commonly referred to as \'The Case of the Missing Canary\'  by none other than Sir Arthur Conan Doyle, the creator of Sherlock Holmes, but it was later included in the collection "The Case-Book of Sherlock Holmes"  and was adapted into a film called "The Case of the Missing Canary"'}

### As you can see, the query is asking something that does not exist in the document. The LLM responds with information that actually is not true. You don't want this to happen, so you must add a prompt to the LLM.


### Using Prompt Template

In [12]:
# 1. Define custom prompt template
prompt_template = '''Use the information from the document to answer the question at the end.
If you don't know the answer,
just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
'''

# 2. Create a PromptTemplate object from the string
prompt = PromptTemplate(template=prompt_template,
                        input_variables=['context', 'question']) # Note: 'question' is standard

# 3. Create the dictionary to pass the custom prompt
chain_type_kwargs = {'prompt': prompt}



In [13]:
# 4. Use the dictionary when initializing your RetrievalQA chain
qa = RetrievalQA.from_chain_type(llm = llama_llm,
                                 chain_type = 'stuff',
                                 retriever = docsearch.as_retriever(),
                                 chain_type_kwargs = chain_type_kwargs,
                                 return_source_documents = False
                                 )
query = 'Is there a movie based on the book?'
qa.invoke(query)

{'query': 'Is there a movie based on the book?',
 'result': 'I don\'t know. The provided text does not mention any information about a movie adaptation of the book. However, I can suggest that the book "The Hound of the Baskervilles" by Sir Arthur Conan Doyle is a famous mystery novel featuring Sherlock Holmes, and it has been adapted into numerous films and other media. But the text provided does not give any specific information about a movie adaptation.'}

From the answer, you can see that the model responds with "don't know".


In [14]:
# --- Gradio Application Functions ---
# These functions will be called by the Gradio interface.
def answer_question(query):
    """
    This function processes the user's query and returns an answer.
    """
    if not query:
        return "Please enter a question."

    try:
        response = qa.invoke({'query': query})
        return response['result']
    except Exception as e:
        return f"An error occurred: {e}"

In [16]:
# --- Gradio Interface ---
# Define the Gradio web interface
import gradio as gr
rag_application = gr.Interface(
    fn=answer_question,
    allow_flagging='never',
    inputs=gr.Textbox(label='Input Query', lines=5, placeholder="Enter your question here"),
    outputs=gr.Textbox(label="Answer"),
    title="Groq-powered RAG Chatbot for Sherlock Holmes",
    description="Ask a question and get an answer based on 'The Adventures of Sherlock Holmes'.",
)

if __name__ == "__main__":
    rag_application.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ace2ba77e635e6aa32.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
